In [ ]:
%pylab inline
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# install OpenCV
!pip install opencv-python

# the usual ...
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from skimage import img_as_float
import cv2
import math


In [ ]:
# read an image (make sure CavePainting.jpg is in the main folder in your Google Drive)
img = cv2.imread('/content/drive/My Drive/Colab Notebooks/Assignments/A2/ex1.jpg') # READS IN NUMPY ARRAY
#convert color
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)
print(img.shape)

In [179]:
#@title Part 2: Image Resizing with Seam Carving
def GradientMag(img):
  img = img.astype('float32')
  blur = cv2.GaussianBlur(img, (3, 3), 0, 0)
  gray = cv2.cvtColor(blur, cv2.COLOR_BGR2GRAY)
  sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3, scale=1, delta=0, borderType=cv2.BORDER_DEFAULT)
  sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3, scale=1, delta=0, borderType=cv2.BORDER_DEFAULT)
  ENERGY = cv2.add(np.absolute(sobelx), np.absolute(sobely))
  return ENERGY

def Search_seam(img, gradient):
  # Find seam with Dynamic programming
  # Returns min_col(the index at the last row w/ minimum sum of energy), and the matrix sum_energy 
  rows, columns = gradient.shape
  #sum_energy is where we store the current sum of energy(DP method)
  sum_energy = np.zeros((rows, columns))
  #copy first row of energy matrix to sum_energy
  sum_energy[0,:] = gradient[0,:]
  index_mat = np.zeros((rows, columns))

  #update sum_energy matrix after the 1st row:
  for i in range(1, rows):  
      for j in range(columns):       
        mid = sum_energy[i-1,j]
        if j == 0:
          left = float('inf')
          right = sum_energy[i-1,j+1]
        elif j == columns -1:
          left = sum_energy[i-1,j-1]
          right = float('inf')
        #analyse
        if left == min(left,mid,right):
          index_mat[i,j] = j-1
        elif mid == min(left,mid,right):
          index_mat[i,j] = j
        else:
          index_mat[i,j] = j+1

  
  # Take last row and finding smallest sum of energy: 
  min_energy = float('inf')
  min_col = 0
  for c in range(columns):
    if sum_energy[rows-1,c] < min_energy:
      min_energy = sum_energy[rows-1,c]
      min_col = c
  return  min_col, index_mat


def Remove_path(img, min_col, index_mat):
  #Returns the image with the vertical seam removed by using backtrace, given the 
  #index of the last row(min_col) and the memory matrix(sum_energy)
  print('removed col is, ',min_col)
  rows = img.shape[0]
  cols = img.shape[1]
  result_img = np.zeros((rows, cols-1, 3))

  #Backtrace
  for i in range(rows-1, -1, -1):   
    result_img[i,:,0] = np.delete(img[i,:,0], min_col) 
    result_img[i,:,1] = np.delete(img[i,:,1], min_col) 
    result_img[i,:,2] = np.delete(img[i,:,2], min_col) 
    #find next index to delete
    index = index_mat[i,min_col]
    min_col = index

  return result_img


In [ ]:
def Remove_Col(img, num_removed_col):
  img = img_as_float(img)
  # Return an img with num_removed_col columns removed
  for i in range(num_removed_col): 
    Gxy = GradientMag(img)
    min_col, sum_energy = Search_seam(img, Gxy)
    print('sum energy is',sum_energy)
    #print("iteration i and minIndex is", i , min_col)
    #img = Find_seam_path(img, min_col, sum_energy)
    removed_img = Remove_path(img, min_col, sum_energy)
    img = removed_img
    print("result img's shape", img.shape)

  return img

In [ ]:
image1 = Remove_Col(img, 2)
plt.imshow(image1)